# 1. Prepare Data

In [1]:
import os
import tensorflow as tf

In [2]:
genres = ['Drama', 'Adventure', 'Action', 'Crime', 'Comedy', 'Thriller', 'Animation', 'Mystery', 'Romance', 'Biography']
budget_lst = [
        "under 1 million USD",
        "1-10 million USD",
        "10-25 million USD",
        "25-50 million USD",
        "50 million USD or more"
    ]
def split_label(path):
    filename = os.path.basename(path)
    label_part = filename.strip()[0:16]  # Assuming labels are in the first 16 chars of the filename
    genre_bits = label_part[:10]  # First 10 bits for genres
    budget_bits = label_part[11:16]  # Next 5 bits for budgets

    # Convert string to integers
    genres_label = [int(bit) for bit in genre_bits]
    budget_label = [int(bit) for bit in budget_bits]  # Assuming one-hot encoded, find index of '1'

    return genres_label, budget_label


def decode_label(label_part):
    genres_label, budget_label = label_part
    genres_label = [genres[i] for i in range(len(genres_label)) if genres_label[i] == 1]
    budget_label = [budget_lst[i] for i in range(len(budget_label)) if budget_label[i] == 1]
    

    return {
        'genres': genres_label,
        'budget': budget_label
    }
example_path = "./database/1110000000 00100 10152.jpg"
example_label = split_label(example_path)
print(example_label)
print(decode_label(example_label))



([1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0])
{'genres': ['Drama', 'Adventure', 'Action'], 'budget': ['10-25 million USD']}


In [3]:
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image /= 255.0  # Normalize the image to [0, 1] range
    return image, label

def create_dataset_from_directory(directory="database"):
    paths, genre_labels, budget_labels = [], [], []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg"):
                path = os.path.join(root, file)
                paths.append(path)
                genre_label, budget_label = split_label(path)
                genre_labels.append(genre_label)
                budget_labels.append(budget_label)
    
    # Convert lists to tensor slices
    path_ds = tf.data.Dataset.from_tensor_slices(paths)
    genre_label_ds = tf.data.Dataset.from_tensor_slices(genre_labels)
    budget_label_ds = tf.data.Dataset.from_tensor_slices(budget_labels)

    # Zip the datasets
    label_ds = tf.data.Dataset.zip((genre_label_ds, budget_label_ds))
    image_label_ds = tf.data.Dataset.zip((path_ds, label_ds))

    # Map the load and preprocess function
    image_label_ds = image_label_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return image_label_ds

# Example of usage
dataset = create_dataset_from_directory()
dataset = dataset.shuffle(1000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Output for inspection
for images, labels in dataset.take(1):  # Take 1 batch
    print(images.shape)  # Output the shape of images, e.g., (32, 224, 224, 3)
    print(labels)        # Output the corresponding labels


# Split to train valid test
train_size = int(0.6 * len(dataset))
valid_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - valid_size
train_dataset = dataset.take(train_size)
valid_dataset = dataset.skip(train_size).take(valid_size)
test_dataset = dataset.skip(train_size + valid_size).take(test_size)
print(len(train_dataset), len(valid_dataset), len(test_dataset))


(32, 224, 224, 3)
(<tf.Tensor: shape=(32, 10), dtype=int32, numpy=
array([[1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0

2024-04-19 11:19:15.313838: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# 2. Define Model

In [4]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import layers, models

In [5]:
from tensorflow.keras import layers, models

def build_multi_output_model(num_genres=10, num_budgets=5, input_shape=(224, 224, 3)):
    # Define the layers.
    input_layer = layers.Input(shape=input_shape)
    
    # Add some convolutional layers as an example
    x = layers.Conv2D(32, kernel_size=(3, 3), activation='relu')(input_layer)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    # Flatten the output
    x = layers.Flatten()(x)
    
    # Dense layer with dropout
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    
    # Output layer for genres - assuming multi-label classification (binary for each genre)
    genre_output = layers.Dense(num_genres, activation='sigmoid', name='genre_output')(x)
    
    # Output layer for budget - assuming multi-class classification
    budget_output = layers.Dense(num_budgets, activation='softmax', name='budget_output')(x)
    
    # Construct the model
    model = models.Model(inputs=input_layer, outputs=[genre_output, budget_output])
    
    # Compile the model with appropriate loss functions for each output
    model.compile(optimizer='adam', 
                  loss={'genre_output': 'binary_crossentropy', 'budget_output': 'categorical_crossentropy'},
                  metrics={'genre_output': 'accuracy', 'budget_output': 'accuracy'})
    
    return model

# Create the model
multi_output_model = build_multi_output_model()

# Print model summary to see the full architecture
multi_output_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 222, 222,  │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 111, 111,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 54, 54,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 186624)    │          0 │ max_pooling2d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │ 95,552,000 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ genre_output        │ (None, 10)        │      5,130 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ budget_output       │ (None, 5)         │      2,565 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 95,579,087 (364.61 MB)

 Trainable params: 95,579,087 (364.61 MB)

 Non-trainable params: 0 (0.00 B)

# 3. Model Training

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [7]:
# Callback to save the best model
checkpoint = ModelCheckpoint(
    'best_model.keras',
    save_best_only=True, 
    monitor='val_loss', 
    mode='min'
)

# Callback to reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=5, 
    min_lr=0.001
)

# Callback for early stopping
early_stopping = EarlyStopping(
    monitor='val_genre_output_accuracy',  # Adjusted to the correct metric name
    patience=5, 
    verbose=1, 
    mode='max', 
    restore_best_weights=True
)


# Assuming the model and datasets are prepared correctly
multi_output_model = build_multi_output_model()

# Train the model with added early stopping
history = multi_output_model.fit(
    train_dataset,
    epochs=50,
    validation_data=valid_dataset,
    callbacks=[checkpoint, reduce_lr, early_stopping]
)


Epoch 1/50
277/277 ━━━━━━━━━━━━━━━━━━━━ 272s 975ms/step - budget_output_accuracy: 0.4092 - genre_output_accuracy: 0.6183 - loss: 2.6478 - val_budget_output_accuracy: 0.5476 - val_genre_output_accuracy: 0.7361 - val_loss: 1.5018 - learning_rate: 0.0010
Epoch 2/50
277/277 ━━━━━━━━━━━━━━━━━━━━ 294s 1s/step - budget_output_accuracy: 0.6285 - genre_output_accuracy: 0.6895 - loss: 1.3304 - val_budget_output_accuracy: 0.6399 - val_genre_output_accuracy: 0.7544 - val_loss: 1.2986 - learning_rate: 0.0010
Epoch 3/50
277/277 ━━━━━━━━━━━━━━━━━━━━ 305s 1s/step - budget_output_accuracy: 0.7823 - genre_output_accuracy: 0.6816 - loss: 0.9135 - val_budget_output_accuracy: 0.6732 - val_genre_output_accuracy: 0.7412 - val_loss: 1.2446 - learning_rate: 0.0010
Epoch 4/50
277/277 ━━━━━━━━━━━━━━━━━━━━ 298s 1s/step - budget_output_accuracy: 0.8682 - genre_output_accuracy: 0.6674 - loss: 0.6414 - val_budget_output_accuracy: 0.6933 - val_genre_output_accuracy: 0.7052 - val_loss: 1.2804 - learning_rate: 0.0010
E

# 4. Evaluation and Iteration

In [14]:
results = multi_output_model.evaluate(test_dataset)
total_loss, genre_accuracy, budget_accuracy = results
print(f"Total Loss: {total_loss}")
print(f"Accuracy for Genre Output: {genre_accuracy}")
print(f"Accuracy for Budget Output: {budget_accuracy}")


NameError: name 'results' is not defined